placeholder markdown cell

In [39]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
import os

data = fetch_openml('adult', version=2, as_frame=True)
df = data.frame
df.shape

(48842, 15)

In [40]:

for col in df.select_dtypes(include=['category', 'object']).columns:
    print(f"{col}: {df[col].unique()[:10]} unique values")


workclass: ['Private', 'Local-gov', NaN, 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']
Categories (8, object): ['Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'] unique values
education: ['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', 'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate']
Categories (16, object): ['10th', '11th', '12th', '1st-4th', ..., 'Masters', 'Preschool', 'Prof-school', 'Some-college'] unique values
marital-status: ['Never-married', 'Married-civ-spouse', 'Widowed', 'Divorced', 'Separated', 'Married-spouse-absent', 'Married-AF-spouse']
Categories (7, object): ['Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'] unique values
occupation: ['Machine-op-inspct', 'Farming-fishing', 'Protective-serv', NaN, 'Other-service', 'Prof-specialty', 'Craft-repair', 'Adm-clerical

In [41]:
df.isnull().sum()

age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
class                0
dtype: int64

in the code cell below we are going to check the missing values from above are classified as MCAR, MAR, or MNAR. 

In [42]:
both_missing = df['workclass'].isnull() & df['occupation'].isnull()

print(f"Both missing: {both_missing.sum()} rows")
print(f"Only workclass missing: {(df['workclass'].isnull() & df['occupation'].notna()).sum()} rows")
print(f"Only occupation missing: {(df['workclass'].notna() & df['occupation'].isnull()).sum()} rows")

Both missing: 2799 rows
Only workclass missing: 0 rows
Only occupation missing: 10 rows


From our output above we have ruled out MCAR for missing values, now we will check if it is MAR or MNAR, since the missingness is related to the employment status, which relates to income. 

in the cell below we will run another crosstabulation to verify the missing value classificiation for future use.

In [43]:
df['workclass_missing'] = df['workclass'].isnull().astype(int)
pd.crosstab(df['workclass_missing'], df['class'], normalize='index')

class,<=50K,>50K
workclass_missing,,
0,0.751928,0.248072
1,0.905323,0.094677


In [44]:
'''
checking 'native-country' column for missing values to verify if they are random
'''
df['native-country_missing'] = df['native-country'].isnull().astype(int)
pd.crosstab(df['native-country_missing'], df['class'], normalize='index')

print(f"native-country missing AND workclass missing: {(df['native-country'].isnull() & df['workclass'].isnull()).sum()}")
print(f"native-country missing AND workclass filled: {(df['native-country'].isnull() & df['workclass'].notna()).sum()}")

native-country missing AND workclass missing: 46
native-country missing AND workclass filled: 811


In [45]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'class', 'workclass_missing', 'native-country_missing'],
      dtype='object')

In [46]:
pd.crosstab(df['native-country_missing'], df['class'], normalize='index')

class,<=50K,>50K
native-country_missing,,
0,0.761029,0.238971
1,0.743291,0.256709


'workclass' column has 2799 missing values, with a MAR/MNAR pattern that is correlated with low income, overlaps with occupation.
'occupation' column has 2809 missing values, with a MAR/MNAR pattern that is also correlated with low income, overlaps with workclass.
'native-country' column has 857 missing values, with a MCAR pattern that has no income correlation.

for this project, due to the nature of the analysis and training a model later on, I will be dropping the rows, in order to not add any more unnessecary complexity to the model training.

next we will copy the dataframe, dropping the missing values from the 'class', 'occupation', and 'native-country' columns

In [47]:
df_clean = df.dropna(subset=['workclass', 'occupation', 'native-country']).copy()
print(f"Dropped {len(df) - len(df_clean)} rows with missing values.")
print(f"Remaining: {len(df_clean)} rows.")

# dropping the temp columns we created earlier.
df_clean = df_clean.drop(columns=['workclass_missing', 'native-country_missing'])

Dropped 3620 rows with missing values.
Remaining: 45222 rows.


we are going to save the new cleaned dataset to CSV inside the /data/ folder in the repo, for later use

In [49]:
df_clean.to_csv('../data/adult_clean.csv')
